In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lec_act_9_simulation.ipynb")

# Iterative systems part II

This is a continuation/extension of week 4's simulation with some slight modifications

- Full 2D - position and velocity of a point in space
- Integrate velocity as well as position

Slides: https://docs.google.com/presentation/d/1ruu1Lq9MpYSHiVa5RNrvyxap4yXpTpiXf_VTzHKAFb8/edit?usp=sharing

Code structure: I've broken the code up in to several pieces: Code that is shared across the lecture activity, lab, and homework is in **pinball_routines.py**. Code that is specific to just the one assignment (running the simulation, plotting) is in the JN.

In [ ]:
# The usual imports
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports from week 6 - you don't have to use the matrices from week 6, but if you decide to, here's the code
#. that supports that
import os
import sys
if "Week_6_matrices" not in sys.path:
    sys.path.insert(0, os.path.abspath('../Week_6_matrices'))
print(f"Current path {sys.path}")


In [ ]:
# Do the import of your pinball routines
# 
from pinball_routines import acceleration_due_to_gravity, compute_next_step


# One time step
TODO Compute one time step of the simulation (edit **compute_next_step** in pinball_routines.py)

In [ ]:
# Time step
delta_t = 0.1

starting_state = np.zeros([6, 2])  # meters
starting_state[0, :] = [0, 0] # Start at zero, zero
# Velocity - mostly up with a bit of x 
starting_state[1, :] = [-0.25, 5.0]
# Acceleration is really boring
starting_state[2, :] = [0.0, acceleration_due_to_gravity()]

first_time_step = compute_next_step(starting_state, delta_t=delta_t)
print(f"Checking first time step {first_time_step}")

assert(np.all(np.isclose(first_time_step[0, :], starting_state[0, :] + delta_t * starting_state[1, :])))
assert(np.all(np.isclose(first_time_step[1, :], starting_state[1, :] + delta_t * starting_state[2, :])))

In [ ]:
grader.check("compute_next_time_step")

# Simulation

Note that you could use either the number of time steps OR total time for the last parameter

Timesteps is a bit safer because at least you know it will only go for so many time steps...

In this activity we'll just loop over n time steps; we'll do something "smarter" in the lab

TODO: Fill in ret_pose_all

In [ ]:
# EXAMPLE CODE
# See Iterate n times in Lab 5 ODE systems for an example of how to store values in an array during a simulation time step

# If you have a 2 x n array this works
new_xy = np.array([0.1, 0.2])

array_to_store_in = np.zeros((2, 10))

# Copies both x and y into the first column of array_to_store_in
array_to_store_in[:, 0] = new_xy

next_xy = np.array([0.3, 0.4])
# ... and into the second column
array_to_store_in[:, 1] = new_xy


In [ ]:
def calculate_n_time_steps(starting_state, delta_t=0.1, n_time_steps=100):
    """ Call compute one time step multiple times and store it in a numpy array
    @param starting_state - the starting positino, velocity, acceleration
    @param delta_t - the time step to use. Define a default t value that you've determined works well
    @param n_time_steps - how many time steps to take. Again, default to a reasonable number
    @return position values as a 2xtimesteps numpy array
    """

    # The returned array. We know the size, so we can pre-allocate it
    ret_pose_all = np.zeros((2, n_time_steps))

    # TODO: for the given number of time steps, call compute_next_step and save the position
    # Note: compute_next_step is in pinball_routines.py
    ...
    # All done - return the numpy array
    return ret_pose_all

In [ ]:
# Actually run the simulation
# Time step
delta_t = 0.1

starting_state = np.zeros([3, 2])  # meters
starting_state[0, :] = [0, 0] # Start at zero, zero
# Velocity - mostly up with a bit of x
starting_state[1, :] = [-0.25, 5.0]
# Acceleration is really boring
starting_state[2, :] = [0.0, acceleration_due_to_gravity()]

ret_poses = calculate_n_time_steps(starting_state, delta_t=delta_t, n_time_steps=15)
print(f"Last pose: {ret_poses[:, -1]}")
assert(np.isclose(ret_poses[0, -1], -0.35))
assert(np.isclose(ret_poses[1, -1], -1.918))

In [ ]:
grader.check("simulate")

# Plotted result

TODO: Change delta t and number of time steps so the spacing is closer together and the simulation is stopped shortly after the ball passes the y = 0 line (we'll do this "right" in the lab)

In [ ]:
# This is pretty arbitrary - but I chose to ask the person calling the function to pass in the poses returned from
#  the interation and the initial velocity (so we can see it).
def plot_results(axs, ret_poses, initial_vel, total_time):
    """ plot the results of running the system AND the "correct" closed form result
    @param ret_poses - x y position values in a 2xn numpy array
    @param initial_vel - Show the initial velocity
    @param total_time - the total time the system ran (for closed form solution, delta_t * n time steps)
    @return Nothing
    """
    # The values we calculated in calculate_n_time_steps
    axs.plot((ret_poses[0, 0], ret_poses[0, 0] + initial_vel[0]),
             (ret_poses[1, 0], ret_poses[1, 0] + initial_vel[1]),
             '-m', label="Initial vel")
    axs.plot(ret_poses[0, 0], ret_poses[1, 0], 'xr', markersize=10, label="Start")
    axs.plot(ret_poses[0, :], ret_poses[1, :], '-Xk', label="Poses")

    axs.axis('equal')
    axs.set_title(f"Path of pinball, 0-{total_time} s")
    axs.legend()

In [ ]:
# Time step - change to put spacing closer
# Re-declaring these variables for this problem
delta_t = 0.1

starting_state = np.zeros([6, 2])  # meters
starting_state[0, :] = [0, 0] # Start at zero, zero
# Velocity - mostly up with a bit of x
starting_state[1, :] = [-0.25, 5.0]
# Acceleration is really boring
starting_state[2, :] = [0.0, acceleration_due_to_gravity()]

nrows = 1
ncols = 1
fig, axs = plt.subplots(nrows, ncols, figsize=(4, 4))

# TODO change delta_t and n_time_steps so the simulation ends shortly after it crosses the y = 0 line
#   You will need to over-ride the default parameters for delta t and n time steps
ret_poses = calculate_n_time_steps(starting_state)

# TODO Fix this so that you pass the remaining correct variables (replace each _ with the correct variable)

plot_results(axs, _, _, _)

In [ ]:
grader.check("Plot")

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS TCW3 (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

In [ ]:
grader.check("hours_collaborators")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit through gradescope, Lecture activity 9 Simulation

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)